In [1]:
import numpy as np

from keras import backend as K
K.set_image_dim_ordering('th') # ensure our dimension notation matches

from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D , Convolution2D, AveragePooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD,Adam
from keras.datasets import mnist
from PIL import Image,ImageOps
import argparse
import math
import os
import os.path
import glob

Using TensorFlow backend.


In [2]:
def get_discriminator():
    model = Sequential()
    model.add(Convolution2D(64,5,5,border_mode = "same",input_shape =(1,128,128)))
    model.add(Activation('tanh'))
    model.add(AveragePooling2D(pool_size=(4,4)))
    model.add(Convolution2D(128,5,5))
    model.add(Activation('tanh'))
    model.add(AveragePooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print(get_discriminator().summary)

<bound method Container.summary of <keras.models.Sequential object at 0x10e01b978>>


/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(1, 128, 1..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5))`
  


In [3]:
print(get_discriminator().summary)

<bound method Container.summary of <keras.models.Sequential object at 0x11fb21160>>


/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(1, 128, 1..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5))`
  


In [4]:
def get_generator():
    model = Sequential()
    model.add(Dense(input_dim=100,output_dim = 1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*8*8))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((128,8,8),input_shape = (128*8*8,)))
    model.add(UpSampling2D(size=(4,4)))
    model.add(Convolution2D(64,5,5,border_mode="same"))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(4,4)))
    model.add(Convolution2D(1,5,5,border_mode="same"))
    model.add(Activation('tanh'))
    
    
    return model

In [5]:
def GAN(generator,discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    
    return model

In [6]:
def format_data(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0],width*shape[1]),dtype=generated_images.dtype)
    
    for index,img in enumerate(generated_images):
        i = int(index/width)
        j = index %width
        image[i*shape[0]:(i+1)*shape[0],j*shape[1]:(j+1)*shape[1]] = img[0,:,:]
    
    return image

In [7]:
def load_data(pixels = 128):
    print('Loading data')
    X_train = []
    paths = glob.glob(os.path.normpath(os.getcwd()+"/logos/*.jpg"))
    loaded_count = 0
    
    for path in paths:
        image = Image.open(path)
        image = ImageOps.fit(image,(pixels,pixels),Image.ANTIALIAS)
        image = ImageOps.grayscale(image)
        image = np.asarray(image)
        X_train.append(image)
        
        loaded_count += 1
        
        if loaded_count%10 ==0:
            print('Loaded '+str(loaded_count)+ " images ")
        
    print("Finished loading data")
        
    return np.array(X_train)


In [8]:
np.shape(load_data())

Loading data
Loaded 10 images 
Loaded 20 images 
Loaded 30 images 
Finished loading data


(38, 128, 128)

In [9]:
def train_GAN(epochs,batch_size,load_weights= False,learning_rate = .0005):
    """
    :param epochs: Number of training epochs
    :param batch_size: size of minibatch
    :param load_weights: If true, load trained weights from file, else train from scratch
    """
    X_train = load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
    
    discriminator = get_discriminator()
    generator = get_generator()
    
    if load_weights:
        generator.load_weights('goodgenerator.h5')
        discriminator.load_weights('gooddiscriminator.h5')
        
    GAN_model = GAN(generator,discriminator)
    
    d_optimizer = SGD(lr=learning_rate,momentum=0.9,nesterov=True)
    g_optimizer = SGD(lr=learning_rate,momentum=0.9,nesterov=True)
    
    generator.compile(loss ="binary_crossentropy",optimizer="SGD")
    GAN_model.compile(loss="binary_crossentropy",optimizer=g_optimizer)
    discriminator.trainable = True
    discriminator.compile(loss="binary_crossentropy",optimizer=d_optimizer)
    
    noise = np.zeros((batch_size,100))
    
    for epoch in range(epochs):
        for index in range(int(X_train.shape[0]/batch_size)):
            
            #generate a batch of random noise to generator input
            for i in range(batch_size):
                noise[i,:] = np.random.uniform(-1,1,100)
                
            image_batch = X_train[index*batch_size:(index+1)*batch_size]
            generated_images = generator.predict(noise,verbose = 0)
            
            # once in a while write generated images
            if index % 20 == 0 and epoch % 10 == 0:
                image = format_data(generated_images)
                image = image*127.5+127.5
                destpath = os.path.normpath(os.getcwd()+ "/logo-generated-images/"+str(epoch)+"_"+str(index)+".png")
                Image.fromarray(image.astype(np.uint8)).save(destpath)
            
            # create a train batch containing generated and real images together
            X = np.concatenate((image_batch,generated_images))
            y = [0.9] * batch_size + [0] * batch_size #use 0.9 instead of 1 for label smoothing
            
            #train discriminator
            d_loss = discriminator.train_on_batch(X,y)
            
            # Generate generator training set
            ### generate images from random noise
            for i in range(batch_size):
                noise[i,:] = np.random.uniform(-1,1,100)
                
           ### generate labels
            g_y = [1] * batch_size
            
            discriminator.trainable = False
            g_loss = GAN_model.train_on_batch(noise,g_y)
            discriminator.trainable = True
            
            if epoch % 10 == 0:
                generator.save_weights('goodgenerator.h5', True)
                discriminator.save_weights('gooddiscriminator.h5', True)

In [10]:
def clean_image(image):
    for i in range(1,image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if image[i][j] + image[i+1][j] + image[i][j+1] + image[i-1][j] + image[i][j-1] > 127 * 5:
                image[i][j] = 255
    
    return image

In [17]:
def generate_image_batch(batch_size):
    generator  = get_generator()
    generator.compile(loss="binary_crossentropy",optimizer="SGD")
    generator.load_weights('goodgenerator.h5')
    
    noise = np.zeros((batch_size,100))
    a = np.random.uniform(-1, 1, 100)
    b = np.random.uniform(-1, 1, 100)
    grad = (b - a) / batch_size
    
    for i in range(batch_size):
        noise[i, :] = np.random.uniform(-1, 1, 100)
    generated_images = generator.predict(noise, verbose=1)
    
    print(generated_images.shape)
    for image in generated_images:
        image = image[0]
        image = image*127.5+127.5
        Image.fromarray(image.astype(np.uint8)).save("dirty.png")
        Image.fromarray(image.astype(np.uint8)).show()
        clean_image(image)
        image = Image.fromarray(image.astype(np.uint8))
        image.show()        
        image.save("clean.png")

In [12]:
train_GAN(5,10,False)

Loading data
Loaded 10 images 
Loaded 20 images 
Loaded 30 images 
Finished loading data


/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(1, 128, 1..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5))`
  
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/Users/ll

In [19]:
generate_image_batch(10)

/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/Users/llealhernandez/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (5, 5), padding="same")`
  del sys.path[0]


10/10 [==============================] - 0s
(10, 1, 128, 128)
